The Canada Warbler is a species of Warbler that migrates through Pennsylvania each Spring. It travels from it's wintering habitat in South America to the boreal swamps of Canada to breed. The species is of interest to birdwatchers and photographers due to its bright yellow plumage and distinct black necklace. The dataset below contains data from eBird for all Canada Warbler sightings in Pennsylvania from April 2010 to June 2020. I will use this data to map out migration patterns across the state and identify regions with the most sightings. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas
from pandas import Series, DataFrame
import plotly.express as px

Pull in .csv data file containing all Canada Warbler sighting data in Pennsylvania from April 2010 to June 2020.

In [ ]:
data = pd.read_csv(r'C:\Users\mikea\Documents\Python Practice\eBird_Geograpic_Analysis_Canada_Warbler\Data\canada_warbler_PA_2010_2020.csv')

Clean up the data by removing spaces in headers and converting headers to lowercase, then show column names.

In [ ]:
data.columns = [x.lower() for x in data.columns]
data.columns = data.columns.str.replace(' ', '_')

Create a consolidated dataframe with only the data we care about.

In [ ]:
consolidated_data = data[[
    'global_unique_identifier',
    'county', 'county_code', 'observation_count',
    'locality', 'locality_id', 'locality_type',
    'latitude', 'longitude', 'observation_date']].copy()
consolidated_data.head()
consolidated_data = consolidated_data[consolidated_data.observation_count != 'X']

Create new columns with date specific attributes so we can filter out data more easily

In [ ]:
import calendar
consolidated_data['month'] = pd.DatetimeIndex(consolidated_data['observation_date']).month
consolidated_data['year'] = pd.DatetimeIndex(consolidated_data['observation_date']).year
consolidated_data['week'] = pd.DatetimeIndex(consolidated_data['observation_date']).week
consolidated_data['day'] = pd.DatetimeIndex(consolidated_data['observation_date']).day

Let's take a look at the data for May 2020. We'll create a new dataframe and add a column with geometry using geopandas.

In [ ]:
may_2020_cw_all_data = consolidated_data.query('month =="5" & year =="2020"')
geographical_data_frame_may_2020 = geopandas.GeoDataFrame(
    may_2020_cw_all_data, geometry=geopandas.points_from_xy(may_2020_cw_all_data.longitude, may_2020_cw_all_data.latitude))

To graph this data, we'll need to have a shape file containing geographical information for Pennsylvania. I downloaded this .shp from data.gov. This will be used to plot the background map for our data.

Next, we need to know the min/max latitude and longitude values for our sightings. This will give us an idea of where our data lies on the map. Note: these don't tell us the min/max values of the state, just a general idea.

In [ ]:
state_map = geopandas.read_file("C:/Users/mikea/Documents/Python Practice/eBird_Geograpic_Analysis_Canada_Warbler/Data/tl_2016_42_cousub/tl_2016_42_cousub.shp")
print(geographical_data_frame_may_2020.latitude.min())
print(geographical_data_frame_may_2020.latitude.max())
print(geographical_data_frame_may_2020.longitude.min())
print(geographical_data_frame_may_2020.longitude.max())

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
state_map.plot(ax=ax, alpha=0.4,color="grey")
geographical_data_frame_may_2020.plot(column="day",ax=ax,alpha=0.5, legend=True,markersize=10)
plt.title("May 2020 Canada Warbler Sighting Locations by Day of the Month", fontsize=15,fontweight="bold")
plt.xlim(-81,-74.5)
plt.ylim( 39.5,42.8)
plt.grid(True)
plt.show()

To create a bubble map with points with varying marker sizes by count per day at a given location, we need to create a column from the geometry column so a .count() function can be called.

In [ ]:
geographical_data_frame_may_2020['geometry_str'] = geographical_data_frame_may_2020.geometry.astype(str)

Below, we create a new dataframe grouped by coordinates with a count of sightings at each coordinate for all of May.

In [ ]:
may_2020_count_per_location = geographical_data_frame_may_2020.groupby([
    "geometry_str", "locality", "latitude", "longitude", "county", "county_code"
    ]).size().reset_index(name="sighting_count")

In order to convert the geometry_str column back into a geopandas geometry, we'll do the following:

In [ ]:
from shapely import wkt

may_2020_count_per_location["coordinates"] = may_2020_count_per_location["geometry_str"].apply(wkt.loads)
gdf_may_2020_count_per_location = geopandas.GeoDataFrame(may_2020_count_per_location, geometry="coordinates")
gdf_may_2020_count_per_location = gdf_may_2020_count_per_location.drop(columns=["geometry_str"])

Now we can use a bubble plot with marker size set to the count of sightings at each location. This gives us an idea of where some of the best hotspots in the state for Canada Warbler are, at a very high level view anyway.

In [ ]:
fig, ax = plt.subplots(figsize=(16,16))
state_map.plot(ax=ax, alpha=0.4,color="grey")
gdf_may_2020_count_per_location.plot(
    ax=ax,color="#07424A", markersize="sighting_count",alpha=0.9, categorical=False, legend=True )
plt.xlim(-81,-74.5)
plt.ylim( 39.5,42.8)
plt.grid(True)
plt.show()

Now it's time to create an interactive visualization, where the number of sightings also has a color component.

In [ ]:
gdf_may_2020_count_per_location['coordinates'] = gdf_may_2020_count_per_location['coordinates'].centroid

In [ ]:
fig = px.scatter_mapbox(
    gdf_may_2020_count_per_location, 
        lat="latitude", 
                        lon="longitude", color="sighting_count", size="sighting_count", hover_name="locality", zoom=5.8)
fig.update_layout(mapbox_style="open-street-map")
fig.show()

Instead of doing analysis for only May 2020, would be more inightful to have an interactive plot with a time scroller. To do this, I'll create a new gdf dataframe from the initial 'consolidated_data' dataframe, but this time I won't filter by date.

In [ ]:
gdf_cw_all_data = geopandas.GeoDataFrame(
    consolidated_data, geometry=geopandas.points_from_xy(consolidated_data.longitude, consolidated_data.latitude))

In [ ]:
gdf_cw_all_data['geometry_str'] = gdf_cw_all_data.geometry.astype(str)


In [ ]:
gdf_cw_all_data_grouped = gdf_cw_all_data.groupby(["geometry_str", "locality", "latitude", "longitude", "county", "county_code", "observation_date"]).size().reset_index(name="sighting_count")

In [ ]:
gdf_cw_all_data_grouped["coordinates"] = gdf_cw_all_data_grouped["geometry_str"].apply(wkt.loads)
gdf_cw_all_data_grouped = geopandas.GeoDataFrame(gdf_cw_all_data_grouped, geometry="coordinates")
gdf_cw_all_data_grouped['observation_date'] = pd.to_datetime(gdf_cw_all_data_grouped['observation_date'].str.strip(), format='%m/%d/%Y')

In [ ]:
gdf_cw_all_data_grouped['observation_date'] = gdf_cw_all_data_grouped['observation_date'].dt.strftime('%Y/%m/%d')
gdf_cw_all_data_by_date = gdf_cw_all_data_grouped.iloc[gdf_cw_all_data_grouped.observation_date.sort_values().index]

In [ ]:
min_date = gdf_cw_all_data_by_date.observation_date.min()
max_date = gdf_cw_all_data_by_date.observation_date.max()

fig = px.scatter_mapbox(gdf_cw_all_data_by_date, 
                 lat="latitude", lon="longitude", animation_frame="observation_date",
            size="sighting_count", color="sighting_count", hover_name="locality", range_color = (0,20),
           zoom = 5.8)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(xaxis = {'type' : 'date'})
fig.update_layout(xaxis = {'dtick' : 86400000.0 * 30})
fig.update_layout(legend= {'itemsizing': 'constant'})
fig.update_layout(transition = {'duration': 30})
fig.update_xaxes(range=[min_date, max_date])

fig.show()


Now let's build a Choropleth map!

The maps I build below will represent the number of Canada Warbler sightings per county by year, so I'll start by creating a column only containing the year portion of the observation dates

In [ ]:
gdf_cw_all_data_by_date['observation_date'] = pd.to_datetime(gdf_cw_all_data_by_date['observation_date'])
gdf_cw_all_data_by_date['observation_year'] = gdf_cw_all_data_by_date['observation_date'].dt.strftime('%Y')

I've pulled in data from the Homeland Infrastructure Foundation for all of the FIPS codes in Pennsylvania. As with any other .csv file, I perform some basic clean-up steps to tidy up the column headers.

In [ ]:
fips_data = pd.read_csv(r'C:\Users\mikea\Documents\Python Practice\eBird_Geograpic_Analysis_Canada_Warbler\Data\fips_codes_PA.csv')
fips_data.columns = [x.lower() for x in fips_data.columns]
fips_data.columns = fips_data.columns.str.replace(' ', '_')
fips_data = fips_data.rename(columns={"short_county_name": "county"})

The state code (42) is in a separate column than the county codes (3-digits). Since the the geojson file we'll be working with requires the 5-digit FIPS codes to plot the maps, I'll need to concatenate. Pandas removed the leading zeroes from the county FIPs codes, so the first line of code below fills in as needed. 

In [ ]:
fips_data['county_fips_code'] = fips_data['county_fips_code'].apply(lambda x: '{0:0>3}'.format(x))
fips_data['state_fips_code'] = fips_data['state_fips_code'].astype(str)
fips_data["full_fips_code"] = fips_data["state_fips_code"] + fips_data["county_fips_code"]
fips_data = fips_data.set_index("county")
fips_data = fips_data.sort_index()

Now I'll create a small dataframe by dissolving (aka aggregating) data in the gdf_cw_all_data_by_date dataframe by county, county code, and observation year. This will sum up the sighting counts for unique sightings in each county by year.

In [ ]:
data_by_county = gdf_cw_all_data_by_date.dissolve(by = ['county_code', 'county', 'observation_year'], aggfunc = 'sum')
data_by_county = data_by_county.drop(columns=["longitude", "latitude"])
data_by_county = data_by_county.reset_index()

To plot this data, we'll first need to merge the dataframe containing the FIPS code with the aggregated dataframe containing the bird sightings. To do this, I create a new dataframe containing only FIPS info and county name to prepare for the merge. I will be joining on the name of the county. I noticed while joining that some FIPS codes were showing as NaN after the merge and discovered this was due to white space in the FIPS dataframe. The third line of code removes the white space causing the issue.

In [ ]:
fips_to_merge = fips_data['full_fips_code']
fips_to_merge = fips_to_merge.reset_index()
fips_to_merge['county'] = fips_to_merge['county'].str.strip() #for some reason the fips file had whitespaces, leading to NaN in merge for some counties

Now the dataframes can be merged. To do this we'll use an outer join on the 'county' column.

We need to sort by 'year' here so that any time series animated visualizations display in the proper order.

In [ ]:
merged_data_by_county = pd.merge(data_by_county, fips_to_merge, how='outer', on=['county'])
merged_data_by_county['sighting_count'] = merged_data_by_county['sighting_count'].astype(float)
merged_data_by_county = merged_data_by_county.sort_values(by=['observation_year'])

Now, Plotly Express requires a geojson file to create a map. We'll use the urlopen module from the urllib.request package the json package to bring in a file containin all county FIPS codes in the USA.

In [ ]:
from urllib.request import urlopen
import json

with urlopen("https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json") as response:
 counties = json.load(response)

We can plot this out for a single year, as shown below...

In [ ]:
fig = px.choropleth(merged_data_by_county.query("observation_year == '2011'"), geojson=counties, locations="full_fips_code", color="sighting_count",
 color_continuous_scale="spectral",
 range_color=(1, 150),
 scope="usa",
 labels={"sighting_count":"sighting_count"}
 )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_geos(fitbounds="locations")
fig.show()

Better yet, we can create an animated time-series visualization!

In [ ]:
fig = px.choropleth(merged_data_by_county, geojson=counties, locations="full_fips_code", color="sighting_count",
 color_continuous_scale="spectral",
 animation_frame = "observation_year",
 range_color=(1, 180),
 scope="usa",
 labels={"sighting_count":"sighting_count"},
 hover_name="county"
 )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_geos(fitbounds="locations")
fig.update_layout(transition = {"duration": 1000})
fig.show()